In [74]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [73]:
array([1, 2, 3]) + 1.0

array([2., 3., 4.])

# Introduction

ulab is a C module for micropython. The aim was to implement a small subset of numpy. We chose those functions and methods that might be useful in the context of a microcontroller. This means low-level data processing of linear (array) and two-dimensional (matrix) data.

The main points of ulab are 

- compact, iterable and slicable container of numerical data in 1, and 2 dimensions (arrays and matrices). In addition, these containers support all the relevant unary and binary operators (e.g., `len`, `bool`, ==, +, *, etc.)
- vectorised computations on micropython iterables and numerical arrays/matrices
- basic linear algebra routines (like matrix inversion, matrix reshaping, and transposition)
- polynomial first to numerical data
- fast Fourier transforms

Each section of the implementation part kicks out with a short discussion on what can be done with the module, and what are the tripping points at the C level. I hope that these musings can be used as a starting point for further discussion on the code.

# Environmental settings and magic commands

The C source code, as well as the documentation (mainly verbose comments on certain aspects of the implementation) is contained in this notebook. The code is exported to separate C files in /usermod/ulab/, and then compiled from this notebook. However, I would like to stress that the compilation does not require a jupyter notebook. It can be done from the command line. 

Testing is done on the unix and stm32 ports of micropython, also directly from the notebook. This is why this section contains a couple of magic functions.

In [1]:
%cd ../../micropython/ports/unix/

/home/v923z/sandbox/micropython/v1.11/micropython/ports/unix


In [2]:
from IPython.core.magic import Magics, magics_class, line_cell_magic
from IPython.core.magic import cell_magic, register_cell_magic, register_line_magic
import subprocess
import os

In [3]:
@register_cell_magic
def micropython(line, cell):
    with open('/dev/shm/micropython.py', 'w') as fout:
        fout.write(cell)
    proc = subprocess.Popen(["./micropython", "/dev/shm/micropython.py"], 
                            stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(proc.stdout.read().decode("utf-8"))
    print(proc.stderr.read().decode("utf-8"))
    return None

In [4]:
import IPython

js = """
    (function () {
        var defaults = IPython.CodeCell.config_defaults || IPython.CodeCell.options_default;
        defaults.highlight_modes['magic_text/x-csrc'] = {'reg':[/^\\s*%%ccode/]};
    })();
    """
IPython.core.display.display_javascript(js, raw=True)

js = """
    (function () {
        var defaults = IPython.CodeCell.config_defaults || IPython.CodeCell.options_default;
        defaults.highlight_modes['magic_text/x-csrc'] = {'reg':[/^\\s*%%makefile/]};
    })();
    """
IPython.core.display.display_javascript(js, raw=True)

In [5]:
@magics_class
class MyMagics(Magics):
        
    @cell_magic
    def ccode(self, line, cell):
        copyright = """/*
 * This file is part of the micropython-ulab project, 
 *
 * https://github.com/v923z/micropython-ulab
 *
 * The MIT License (MIT)
 *
 * Copyright (c) 2019 Zoltán Vörös
*/
    """
        if line:
            with open('../../../ulab/code/'+line, 'w') as cout:
                cout.write(copyright)
                cout.write(cell)
            print('written %d bytes to %s'%(len(copyright) + len(cell), line))
            return None

ip = get_ipython()
ip.register_magics(MyMagics)

# Compiling the module

Detailed instructions on how to set up and compile a C module can be found in chapter 2 of https://micropython-usermod.readthedocs.io/en/latest/. 

First, on the command line, you should clone both the micropython, and the `ulab` repositories: 

```bash
git clone https://github.com/micropython/micropython.git
```
Then navigate to your micropython folder, and run 

```bash
git clone https://github.com/v923z/micropython-ulab.git ulab
```

Finally, in the `mpconfigport.h` header file of the port that you want to compile for, you have to define the variable `MODULE_ULAB_ENABLED`

```make
#define MODULE_ULAB_ENABLED (1)
```

At this point, you should be able to run make in the port's root folder:

```bash
make USER_C_MODULES=../../../ulab all
```

# The ndarray type

## General comments

ndarrays are efficient containers of numerical data of the same type (i.e., integers or floats). In addition to storing the actual data, the type definition has three additional members (on top of the `base` type). Namely, two `size_t` objects, `m`, and `n`, which give the dimensions of the matrix (obviously, if the ndarray is meant to be linear, either `m`, or `n` is equal to 1), as well as the byte size, `bytes`, i.e., the number of bytes consumed by the data container. `bytes` is equal to `m*n` for `byte` types (`uint8`, and `int8`), to `m*n*2` for integers (`uint16`, and `int16`), and `m*n*4` for floats. 

The type definition is 

```c
typedef struct _ndarray_obj_t {
    mp_obj_base_t base;
    size_t m, n;
    mp_obj_array_t *data;
    size_t bytes;
} ndarray_obj_t;
```

**NOTE: with a little bit of extra effort, mp_obj_array_t can be replaced by a single void array. We should, perhaps, consider the pros and cons of that. One patent advantage is that we could get rid of the verbatim copy of array_new function in ndarray.c. On the other hand, objarray.c facilities couldn't be used anymore.**

Most of the functions in ndarray.c are internal (i.e., not exposed to the python interpreter). Exception to this rule are the `shape`, `size`, and `rawsize` functions. This latter one is not standard in numpy, and is meant to return the total number of bytes used by the container. Since the RAM of a microcontroller is limited, I deemed this to be a reasonable addition for optimisation purposes, but could later be removed, if it turns out to be of no interest.

## Initialisation

An ndarray can be initialised by passing an iterable (linear arrays), or an iterable of iterables (matrices) into the constructor as

In [205]:
%%micropython

from ulab import ndarray

a = ndarray([1, 2, 3, 4])
print(a)
a = ndarray([[1, 2, 3, 4], [2, 3, 4, 5]])
print(a)
a = ndarray([range(10), range(10)])
print(a)

ndarray([1.0, 2.0, 3.0, 4.0])

ndarray([[1.0, 2.0, 3.0, 4.0],
	 [2.0, 3.0, 4.0, 5.0]])

ndarray([[0.0, 1.0, 2.0, ..., 7.0, 8.0, 9.0],
	 [0.0, 1.0, 2.0, ..., 7.0, 8.0, 9.0]])





In addition, the constructor can take a keyword argument, dtype, that will force type conversion.

## Slicing and subscriptions

Slicing and subscription works only on the flattened array. This has to do with the fact that slices can be given only in the form `myobject[1:10:2]`, and slice objects cannot be separated by commas, i.e., this won't be interpreted:

```python
myobject[1:10:2, 3:10:3]
```

This is something that should be sorted out in the future. As a temporary solution, we will have a special `getter` and `setter` method (see later) that handles this issue.

## Iterators

Flattened `ndarray` objects can be iterated on:

In [746]:
%%micropython

from ulab import ndarray

a = ndarray([[1, 2, 3, 4], [6, 7, 8, 9]])

for _a in a: print(_a)

1.0
2.0
3.0
4.0
6.0
7.0
8.0
9.0




## ndarray.h

In [54]:
%%ccode ndarray.h

#ifndef _NDARRAY_
#define _NDARRAY_

#include "py/objarray.h" // this can in the future be dropped
#include "py/binary.h"   // this can in the future be dropped
#include "py/objstr.h"

#define PRINT_MAX  10


const mp_obj_type_t ulab_ndarray_type;


enum NDARRAY_TYPE {
    NDARRAY_UINT8 = 'b',
    NDARRAY_INT8 = 'B',
    NDARRAY_UINT16 = 'i', 
    NDARRAY_INT16 = 'I',
    NDARRAY_FLOAT = 'f',
};

typedef struct _ndarray_obj_t {
    mp_obj_base_t base;
    size_t m, n;
    size_t len;
    mp_obj_array_t *data;
    size_t bytes;
} ndarray_obj_t;

mp_obj_t mp_obj_new_ndarray_iterator(mp_obj_t , size_t , mp_obj_iter_buf_t *);

float ndarray_get_float_value(void *, uint8_t , size_t );
void ndarray_print_row(const mp_print_t *, mp_obj_array_t *, size_t , size_t );
void ndarray_print(const mp_print_t *, mp_obj_t , mp_print_kind_t );
void ndarray_assign_elements(mp_obj_array_t *, mp_obj_t , uint8_t , size_t *);
ndarray_obj_t *create_new_ndarray(size_t , size_t , uint8_t );
mp_obj_t ndarray_copy(mp_obj_t );
mp_obj_t ndarray_make_new(const mp_obj_type_t *, size_t , size_t , const mp_obj_t *);
mp_obj_t ndarray_subscr(mp_obj_t , mp_obj_t , mp_obj_t );
mp_obj_t ndarray_getiter(mp_obj_t , mp_obj_iter_buf_t *);

mp_obj_t ndarray_shape(mp_obj_t );
mp_obj_t ndarray_size(mp_obj_t , mp_obj_t );
mp_obj_t ndarray_rawsize(mp_obj_t );

#endif

written 1522 bytes to ndarray.h


## ndarray.c

In [163]:
%%ccode ndarray.c

#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "py/runtime.h"
#include "py/binary.h"
#include "py/obj.h"
#include "py/objtuple.h"
#include "ndarray.h"

// This function is copied verbatim from objarray.c
STATIC mp_obj_array_t *array_new(char typecode, size_t n) {
    int typecode_size = mp_binary_get_size('@', typecode, NULL);
    mp_obj_array_t *o = m_new_obj(mp_obj_array_t);
    // this step could probably be skipped: we are never going to store a bytearray per se
    #if MICROPY_PY_BUILTINS_BYTEARRAY && MICROPY_PY_ARRAY
    o->base.type = (typecode == BYTEARRAY_TYPECODE) ? &mp_type_bytearray : &mp_type_array;
    #elif MICROPY_PY_BUILTINS_BYTEARRAY
    o->base.type = &mp_type_bytearray;
    #else
    o->base.type = &mp_type_array;
    #endif
    o->typecode = typecode;
    o->free = 0;
    o->len = n;
    o->items = m_new(byte, typecode_size * o->len);
    return o;
}

float ndarray_get_float_value(void *data, uint8_t typecode, size_t index) {
    if(typecode == NDARRAY_UINT8) {
        return (float)((uint8_t *)data)[index];
    } else if(typecode == NDARRAY_INT8) {
        return (float)((int8_t *)data)[index];
    } else if(typecode == NDARRAY_UINT16) {
        return (float)((uint16_t *)data)[index];
    } else if(typecode == NDARRAY_INT16) {
        return (float)((int16_t *)data)[index];
    } else {
        return (float)((float_t *)data)[index];
    }
}

void ndarray_print_row(const mp_print_t *print, mp_obj_array_t *data, size_t n0, size_t n) {
    mp_print_str(print, "[");
    size_t i;
    if(n < PRINT_MAX) { // if the array is short, print everything
        mp_obj_print_helper(print, mp_binary_get_val_array(data->typecode, data->items, n0), PRINT_REPR);
        for(i=1; i<n; i++) {
            mp_print_str(print, ", ");
            mp_obj_print_helper(print, mp_binary_get_val_array(data->typecode, data->items, n0+i), PRINT_REPR);
        }
    } else {
        mp_obj_print_helper(print, mp_binary_get_val_array(data->typecode, data->items, n0), PRINT_REPR);
        for(i=1; i<3; i++) {
            mp_print_str(print, ", ");
            mp_obj_print_helper(print, mp_binary_get_val_array(data->typecode, data->items, n0+i), PRINT_REPR);
        }
        mp_printf(print, ", ..., ");
        mp_obj_print_helper(print, mp_binary_get_val_array(data->typecode, data->items, n0+n-3), PRINT_REPR);
        for(size_t i=1; i<3; i++) {
            mp_print_str(print, ", ");
            mp_obj_print_helper(print, mp_binary_get_val_array(data->typecode, data->items, n0+n-3+i), PRINT_REPR);
        }
    }
    mp_print_str(print, "]");
}

void ndarray_print(const mp_print_t *print, mp_obj_t self_in, mp_print_kind_t kind) {
    (void)kind;
    ndarray_obj_t *self = MP_OBJ_TO_PTR(self_in);
    mp_print_str(print, "ndarray(");
    if((self->m == 1) || (self->n == 1)) {
        ndarray_print_row(print, self->data, 0, self->data->len);
    } else {
        // TODO: add vertical ellipses for the case, when self->m > PRINT_MAX
        mp_print_str(print, "[");
        ndarray_print_row(print, self->data, 0, self->n);
        for(size_t i=1; i < self->m; i++) {
            mp_print_str(print, ",\n\t ");
            ndarray_print_row(print, self->data, i*self->n, self->n);
        }
        mp_print_str(print, "]");
    }
    // TODO: print typecode
    if(self->data->typecode == NDARRAY_UINT8) {
        printf(", dtype='uint8')\n");
    } else if(self->data->typecode == NDARRAY_INT8) {
        printf(", dtype='int8')\n");
    } if(self->data->typecode == NDARRAY_UINT16) {
        printf(", dtype='uint16')\n");
    } if(self->data->typecode == NDARRAY_INT16) {
        printf(", dtype='int16')\n");
    } if(self->data->typecode == NDARRAY_FLOAT) {
        printf(", dtype='float')\n");
    } 
}

void ndarray_assign_elements(mp_obj_array_t *data, mp_obj_t iterable, uint8_t typecode, size_t *idx) {
    // assigns a single row in the matrix
    mp_obj_t item;
    while ((item = mp_iternext(iterable)) != MP_OBJ_STOP_ITERATION) {
        mp_binary_set_val_array(typecode, data->items, (*idx)++, item);
    }
}

ndarray_obj_t *create_new_ndarray(size_t m, size_t n, uint8_t typecode) {
    // Creates the base ndarray with shape (m, n), and initialises the values to straight 0s
    ndarray_obj_t *ndarray = m_new_obj(ndarray_obj_t);
    ndarray->base.type = &ulab_ndarray_type;
    ndarray->m = m;
    ndarray->n = n;
    mp_obj_array_t *data = array_new(typecode, m*n);
    ndarray->bytes = m * n * mp_binary_get_size('@', typecode, NULL);
    // this should set all elements to 0, irrespective of the of the typecode (all bits are zero)
    // we could, perhaps, leave this step out, and initialise the array, only, when needed
    memset(data->items, 0, ndarray->bytes); 
    ndarray->data = data;
    return ndarray;
}

mp_obj_t ndarray_copy(mp_obj_t self_in) {
    // returns a verbatim (shape and typecode) copy of self_in
    ndarray_obj_t *self = MP_OBJ_TO_PTR(self_in);
    ndarray_obj_t *out = create_new_ndarray(self->m, self->n, self->data->typecode);
    int typecode_size = mp_binary_get_size('@', self->data->typecode, NULL);
    memcpy(out->data->items, self->data->items, self->data->len*typecode_size);
    return MP_OBJ_FROM_PTR(out);
}

STATIC uint8_t ndarray_init_helper(size_t n_args, const mp_obj_t *pos_args, mp_map_t *kw_args) {
    static const mp_arg_t allowed_args[] = {
        { MP_QSTR__array_arr, MP_ARG_REQUIRED | MP_ARG_OBJ, {.u_rom_obj = MP_ROM_PTR(&mp_const_none_obj)}},
        { MP_QSTR_dtype, MP_ARG_KW_ONLY | MP_ARG_OBJ, {.u_rom_obj = MP_ROM_QSTR(MP_QSTR_float)} },
    };

    mp_arg_val_t args[MP_ARRAY_SIZE(allowed_args)];
    mp_arg_parse_all(n_args, pos_args, kw_args, MP_ARRAY_SIZE(allowed_args), allowed_args, args);    
    GET_STR_DATA_LEN(args[1].u_rom_obj, str, str_len);
    if(memcmp(str, "uint8", 5) == 0) {
        return NDARRAY_UINT8;
    } else if(memcmp(str, "uint16", 6) == 0) {
        return NDARRAY_UINT16;
    } else if(memcmp(str, "int8", 4) == 0) {
        return NDARRAY_INT8;
    } else if(memcmp(str, "int16", 5) == 0) {
        return NDARRAY_INT16;
    }
    return NDARRAY_FLOAT;
}

mp_obj_t ndarray_make_new(const mp_obj_type_t *type, size_t n_args, size_t n_kw, const mp_obj_t *args) {
    mp_arg_check_num(n_args, n_kw, 1, 2, true);
    mp_map_t kw_args;
    mp_map_init_fixed_table(&kw_args, n_kw, args + n_args);
    uint8_t dtype = ndarray_init_helper(n_args, args, &kw_args);
    
    size_t len1, len2=0, i=0;
    mp_obj_t len_in = mp_obj_len_maybe(args[0]);
    if (len_in == MP_OBJ_NULL) {
        mp_raise_ValueError("first argument must be an iterable");
    } else {
        len1 = MP_OBJ_SMALL_INT_VALUE(len_in);
    }

    // We have to figure out, whether the first element of the iterable is an iterable itself
    // Perhaps, there is a more elegant way of handling this
    mp_obj_iter_buf_t iter_buf1;
    mp_obj_t item1, iterable1 = mp_getiter(args[0], &iter_buf1);
    while ((item1 = mp_iternext(iterable1)) != MP_OBJ_STOP_ITERATION) {
        len_in = mp_obj_len_maybe(item1);
        if(len_in != MP_OBJ_NULL) { // indeed, this seems to be an iterable
            // Next, we have to check, whether all elements in the outer loop have the same length
            if(i > 0) {
                if(len2 != MP_OBJ_SMALL_INT_VALUE(len_in)) {
                    mp_raise_ValueError("iterables are not of the same length");
                }
            }
            len2 = MP_OBJ_SMALL_INT_VALUE(len_in);
            i++;
        }
    }
    // By this time, it should be established, what the shape is, so we can now create the array
    // set the typecode to float, if the format specifier is missing
    ndarray_obj_t *self = create_new_ndarray(len1, (len2 == 0) ? 1 : len2, dtype);
    iterable1 = mp_getiter(args[0], &iter_buf1);
    i = 0;
    if(len2 == 0) { // the first argument is a single iterable
        ndarray_assign_elements(self->data, iterable1, dtype, &i);
    } else {
        mp_obj_iter_buf_t iter_buf2;
        mp_obj_t iterable2; 

        while ((item1 = mp_iternext(iterable1)) != MP_OBJ_STOP_ITERATION) {
            iterable2 = mp_getiter(item1, &iter_buf2);
            ndarray_assign_elements(self->data, iterable2, dtype, &i);
        }
    }
    return MP_OBJ_FROM_PTR(self);
}

mp_obj_t ndarray_subscr(mp_obj_t self_in, mp_obj_t index, mp_obj_t value) {
    // NOTE: this will work only on the flattened array!
    ndarray_obj_t *self = MP_OBJ_TO_PTR(self_in);
    if (value == MP_OBJ_SENTINEL) { 
        // simply return the values at index, no assignment
#if MICROPY_PY_BUILTINS_SLICE
        if (MP_OBJ_IS_TYPE(index, &mp_type_slice)) {
            mp_bound_slice_t slice;
            mp_seq_get_fast_slice_indexes(self->data->len, index, &slice);
            // TODO: this won't work with index reversion!!!
            size_t len = (slice.stop - slice.start) / slice.step;
            ndarray_obj_t *out = create_new_ndarray(1, len, self->data->typecode);
            int _sizeof = mp_binary_get_size('@', self->data->typecode, NULL);
            uint8_t *indata = (uint8_t *)self->data->items;
            uint8_t *outdata = (uint8_t *)out->data->items;
            for(size_t i=0; i < len; i++) {
                memcpy(outdata+(i*_sizeof), indata+(slice.start+i*slice.step)*_sizeof, _sizeof);
            }
            return MP_OBJ_FROM_PTR(out);
        }
#endif
        // we have a single index, return a single number
        size_t idx = mp_obj_get_int(index);
        switch(self->data->typecode) {
            case NDARRAY_UINT8:
                return MP_OBJ_NEW_SMALL_INT(((uint8_t *)self->data->items)[idx]);
            case NDARRAY_INT8:
                return MP_OBJ_NEW_SMALL_INT(((int8_t *)self->data->items)[idx]);
            case NDARRAY_UINT16:
                return MP_OBJ_NEW_SMALL_INT(((uint16_t *)self->data->items)[idx]);
            case NDARRAY_INT16:
                return MP_OBJ_NEW_SMALL_INT(((int16_t *)self->data->items)[idx]);
            case NDARRAY_FLOAT:
                return mp_obj_new_float(((float_t *)self->data->items)[idx]);
        }
    } else { // do not deal with assignment, bail out
        mp_raise_NotImplementedError("subcript assignment is not implemented for ndarrays");
    }
    return mp_const_none;
}

// itarray iterator

mp_obj_t ndarray_getiter(mp_obj_t o_in, mp_obj_iter_buf_t *iter_buf) {
    return mp_obj_new_ndarray_iterator(o_in, 0, iter_buf);
}

typedef struct _mp_obj_ndarray_it_t {
    mp_obj_base_t base;
    mp_fun_1_t iternext;
    mp_obj_t ndarray;
    size_t cur;
} mp_obj_ndarray_it_t;

mp_obj_t ndarray_iternext(mp_obj_t self_in) {
    mp_obj_ndarray_it_t *self = MP_OBJ_TO_PTR(self_in);
    ndarray_obj_t *ndarray = MP_OBJ_TO_PTR(self->ndarray);
    // TODO: in numpy, ndarrays are iterated with respect to the first axis. 
    size_t iter_end = 0;
    if((ndarray->m == 1) || (ndarray->n ==1)) {
        iter_end = ndarray->data->len;
    } else {
        iter_end = ndarray->m;
    }
    if(self->cur < iter_end) {
        if(ndarray->m == ndarray->data->len) { // we are have a linear array
            // read the current value
            mp_obj_t value;
            value = mp_binary_get_val_array(ndarray->data->typecode, ndarray->data->items, self->cur);
            self->cur++;
            return value;
        } else { // we have a matrix, return the 
            ndarray_obj_t *value = create_new_ndarray(1, ndarray->n, ndarray->data->typecode);
            // copy the memory content here
            uint8_t *tmp = (uint8_t *)ndarray->data->items;
            size_t strip_size = ndarray->n * mp_binary_get_size('@', ndarray->data->typecode, NULL);
            memcpy(value->data->items, &tmp[self->cur*strip_size], strip_size);
            self->cur++;
            return value;
        }
    } else {
        return MP_OBJ_STOP_ITERATION;
    }
}

mp_obj_t mp_obj_new_ndarray_iterator(mp_obj_t ndarray, size_t cur, mp_obj_iter_buf_t *iter_buf) {
    assert(sizeof(mp_obj_ndarray_it_t) <= sizeof(mp_obj_iter_buf_t));
    mp_obj_ndarray_it_t *o = (mp_obj_ndarray_it_t*)iter_buf;
    o->base.type = &mp_type_polymorph_iter;
    o->iternext = ndarray_iternext;
    o->ndarray = ndarray;
    o->cur = cur;
    return MP_OBJ_FROM_PTR(o);
}

mp_obj_t ndarray_shape(mp_obj_t self_in) {
    ndarray_obj_t *self = MP_OBJ_TO_PTR(self_in);
    mp_obj_t tuple[2] = {
        mp_obj_new_int(self->m),
        mp_obj_new_int(self->n)
    };
    return mp_obj_new_tuple(2, tuple);
}

mp_obj_t ndarray_size(mp_obj_t self_in, mp_obj_t axis) {
    ndarray_obj_t *self = MP_OBJ_TO_PTR(self_in);
    uint8_t ax = mp_obj_get_int(axis);
    if(ax == 0) {
        return mp_obj_new_int(self->data->len);
    } else if(ax == 1) {
        return mp_obj_new_int(self->m);
    } else if(ax == 2) {
        return mp_obj_new_int(self->n);
    } else {
        return mp_const_none;
    }
}

mp_obj_t ndarray_rawsize(mp_obj_t self_in) {
    // returns a 5-tuple with the 
    // 
    // 1. number of rows
    // 2. number of columns
    // 3. length of the storage (should be equal to the product of 1. and 2.)
    // 4. length of the data storage in bytes
    // 5. datum size in bytes
    ndarray_obj_t *self = MP_OBJ_TO_PTR(self_in);
    mp_obj_tuple_t *tuple = MP_OBJ_TO_PTR(mp_obj_new_tuple(5, NULL));
    tuple->items[0] = MP_OBJ_NEW_SMALL_INT(self->m);
    tuple->items[1] = MP_OBJ_NEW_SMALL_INT(self->n);
    tuple->items[2] = MP_OBJ_NEW_SMALL_INT(self->bytes);
    tuple->items[3] = MP_OBJ_NEW_SMALL_INT(self->data->len);
    tuple->items[4] = MP_OBJ_NEW_SMALL_INT(mp_binary_get_size('@', self->data->typecode, NULL));
    return tuple;
}

// Binary operations
/*
STATIC uint8_t upcasting(ndarray_obj_t lhs, ndarray_obj_t rhs) {
    // returns the upcast typecode
    // what we have to establish is, whether either of sides has a type code that is 
    // 'larger' than the other side
    uint8_t typecode_l, typecode_r;
    switch(lhs->data->typecode) {
        case 'b':
            typecode_l = (0x01 << 0);
        case 'B':
            typecode_l = (0x01 << 1);
        case 'i':
            typecode_l = (0x01 << 2);
        case 'I':
            typecode_l = (0x01 << 3);
        case 'f':
            typecode_l = (0x01 << 4);
    }
    switch(rhs->data->typecode) {
        case 'b':
            typecode_r = (0x01 << 0);
        case 'B':
            typecode_r = (0x01 << 1);
        case 'i':
            typecode_r = (0x01 << 2);
        case 'I':
            typecode_r = (0x01 << 3);
        case 'f':
            typecode_r = (0x01 << 4);
    }
    // Now we have to collect 25 cases 
    if((typecode_l | typecode_r) == (0x01 << 0)) { // 2 cases
        return 'b';
    } else if((typecode_l | typecode_r) == (0x01 << 1)) { // 2 cases
        return 'B';
    } else if((typecode_l | typecode_r) == (0x01 << 2)) { // 2 casaes
        return 'i';
    } else if((typecode_l | typecode_r) == (0x01 << 3)) { // 2 cases
        return 'I';
    } else if((typecode_l | typecode_r) >= (0x01 << 4)) { // 10 cases
        return 'f';
    } else if((typecode_l | typecode_r) == ((0x01 << 0) | (0x01 << 1)) {
        return 'i';
    }
}

mp_obj_t ulab_ndarray_binary_op_helper(mp_binary_op_t op, mp_obj_t lhs, mp_obj_t rhs) {
    // TODO: support scalar operations
    if (MP_OBJ_IS_TYPE(rhs, &mp_type_int) || MP_OBJ_IS_TYPE(rhs, &mp_type_float)) {
        return MP_OBJ_NULL; // op not supported
    } else if(MP_OBJ_IS_TYPE(rhs, &ulab_ndarray_type)) {
        // At this point, the operands should have the same shape
        ndarray_obj_t *ol = MP_OBJ_TO_PTR(lhs);
        ndarray_obj_t *or = MP_OBJ_TO_PTR(rhs);
        ndarray_obj_t *array;
        if((ol->m != or->m) || (ol->n != or->n)) {
            mp_raise_ValueError("operands could not be broadcast together");
        }
        // do not convert types, if they are identical
        // do not convert either, if the left hand side is a float
        if((ol->data->typecode == or->data->typecode) || ol->data->typecode == NDARRAY_FLOAT) {
            array = ulab_ndarray_copy(ol);
        } else {
            // the types are not equal, we have to do some conversion here
            if(or->data->typecode == NDARRAY_FLOAT) {
                array = ulab_ndarray_copy(ol);
            } else if((ol->data->typecode == NDARRAY_INT16) || (or->data->typecode == NDARRAY_INT16)) {
                array = create_new_ndarray(ol->m, ol->n, NDARRAY_INT16);
            } else if((ol->data->typecode == NDARRAY_UINT16) || (or->data->typecode == NDARRAY_UINT16)) {
                array = create_new_ndarray(ol->m, ol->n, NDARRAY_INT16);
            }
        }
        switch(op) {
            case MP_BINARY_OP_ADD:
                for(size_t i=0; i < ol->data->len; i++) {

                }
                return MP_OBJ_FROM_PTR(array);
                break;
            default:
                break;
        }

    }
}

STATIC mp_obj_t ulab_ndarray_binary_op(mp_binary_op_t op, mp_obj_t lhs, mp_obj_t rhs) {
    ndarray_obj_t *ol = MP_OBJ_TO_PTR(lhs);
    ndarray_obj_t *or = MP_OBJ_TO_PTR(rhs);

    // for in-place operations, we won't need this!!!
    ndarray_obj_t *array = ulab_ndarray_copy(ol);    
    switch (op) {
        case MP_BINARY_OP_EQUAL:
            if(!MP_OBJ_IS_TYPE(rhs, &ulab_ndarray_type)) {
                return mp_const_false;
            } else {
                // Two arrays are equal, if their shape, typecode, and elements are equal
                if((ol->m != or->m) || (ol->n != or->n) || (ol->data->typecode != or->data->typecode)) {
                    return mp_const_false;
                } else {
                    size_t i = ol->bytes;
                    uint8_t *l = (uint8_t *)ol->data->items;
                    uint8_t *r = (uint8_t *)or->data->items;
                    while(i) { // At this point, we can simply compare the bytes, the types is irrelevant
                        if(*l++ != *r++) {
                            return mp_const_false;
                        }
                        i--;
                    }
                    return mp_const_true;
                }
            }
            break;
        case MP_BINARY_OP_ADD:
        case MP_BINARY_OP_MULTIPLY: 
            return MP_OBJ_FROM_PTR(array);
            break;

        default:
            return MP_OBJ_NULL; // op not supported
    }
    }
*/

written 18547 bytes to ndarray.c


# Linear algebra

This module contains very basic matrix operators, such as transposing, reshaping, and inverting. The actual inversion is factored out into a helper function, so that the routine can be re-used in other modules. Also note that inversion is based on the notion of a *small number* (epsilon). During the computation of the inverse, a number is treated as 0, if its absolute value is smaller than epsilon. This precaution is required, otherwise, one might run into singular matrices. 

## linalg.h

In [786]:
%%ccode linalg.h

#ifndef _LINALG_
#define _LINALG_

#include "ndarray.h"

#define SWAP(t, a, b) { t tmp = a; a = b; b = tmp; }
#define epsilon        1e-6

mp_obj_t linalg_transpose(mp_obj_t );
mp_obj_t linalg_reshape(mp_obj_t , mp_obj_t );
mp_obj_t linalg_inv(mp_obj_t );
mp_obj_t linalg_multiply(mp_obj_t , mp_obj_t );

#endif

written 492 bytes to linalg.h


## linalg.c

In [440]:
%%ccode linalg.c

#include <stdlib.h>
#include <string.h>
#include "py/obj.h"
#include "py/runtime.h"
#include "linalg.h"

mp_obj_t linalg_transpose(mp_obj_t self_in) {
    ndarray_obj_t *self = MP_OBJ_TO_PTR(self_in);
    // the size of a single item in the array
    uint8_t _sizeof = mp_binary_get_size('@', self->data->typecode, NULL);
    
    // NOTE: In principle, we could simply specify the stride direction, and then we wouldn't 
    // even have to shuffle the elements. The downside of that approach is that we would have 
    // to implement two versions of the matrix multiplication and inversion functions
    
    // NOTE: 
    // if the matrices are square, we can simply swap items, but 
    // generic matrices can't be transposed in place, so we have to 
    // declare a temporary variable
    
    // NOTE: 
    //  In the old matrix, the coordinate (m, n) is m*self->n + n
    //  We have to assign this to the coordinate (n, m) in the new 
    //  matrix, i.e., to n*self->m + m
    
    // one-dimensional arrays can be transposed by simply swapping the dimensions
    if((self->m != 1) && (self->n != 1)) {
        uint8_t *c = (uint8_t *)self->data->items;
        // self->bytes is the size of the bytearray, irrespective of the typecode
        uint8_t *tmp = (uint8_t *)malloc(self->bytes);
        for(size_t m=0; m < self->m; m++) {
            for(size_t n=0; n < self->n; n++) {
                memcpy(tmp+_sizeof*(n*self->m + m), c+_sizeof*(m*self->n + n), _sizeof);
            }
        }
        memcpy(self->data->items, tmp, self->bytes);
        free(tmp);
    } 
    SWAP(size_t, self->m, self->n);
    return mp_const_none;
}

mp_obj_t linalg_reshape(mp_obj_t self_in, mp_obj_t shape) {
    ndarray_obj_t *self = MP_OBJ_TO_PTR(self_in);
    if(!MP_OBJ_IS_TYPE(shape, &mp_type_tuple) || (MP_OBJ_SMALL_INT_VALUE(mp_obj_len_maybe(shape)) != 2)) {
        mp_raise_ValueError("shape must be a 2-tuple");
    }

    mp_obj_iter_buf_t iter_buf;
    mp_obj_t item, iterable = mp_getiter(shape, &iter_buf);
    uint16_t m, n;
    item = mp_iternext(iterable);
    m = mp_obj_get_int(item);
    item = mp_iternext(iterable);
    n = mp_obj_get_int(item);
    if(m*n != self->m*self->n) {
        // TODO: the proper error message would be "cannot reshape array of size %d into shape (%d, %d)"
        mp_raise_ValueError("cannot reshape array (incompatible input/output shape)");
    }
    self->m = m;
    self->n = n;
    return MP_OBJ_FROM_PTR(self);
}

ndarray_obj_t *invert_matrix(mp_obj_array_t *data, size_t N) {
    // After inversion the matrix is most certainly a float
    ndarray_obj_t *tmp = create_new_ndarray(N, N, NDARRAY_FLOAT);
    // initially, this is the unit matrix: this is what will be returned a
    // after all the transformations
    ndarray_obj_t *unitm = create_new_ndarray(N, N, NDARRAY_FLOAT);

    float *c = (float *)tmp->data->items;
    float *unit = (float *)unitm->data->items;
    mp_obj_t elem;
    float elemf;
    for(size_t m=0; m < N; m++) { // rows first
        for(size_t n=0; n < N; n++) { // columns next
            // this could, perhaps, be done in single line...
            elem = mp_binary_get_val_array(data->typecode, data->items, m*N+n);
            elemf = (float)mp_obj_get_float(elem);
            memcpy(&c[m*N+n], &elemf, sizeof(float));
        }
        // initialise the unit matrix
        elemf = 1.0;
        memcpy(&unit[m*(N+1)], &elemf, sizeof(float));
    }
    for(size_t m=0; m < N; m++){
        // this could be faster with ((c < epsilon) && (c > -epsilon))
        if(abs(c[m*(N+1)]) < epsilon) {
            // TODO: check what kind of exception numpy raises
            mp_raise_ValueError("input matrix is singular");
        }
        for(size_t n=0; n < N; n++){
            if(m != n){
                elemf = c[N*n+m] / c[m*(N+1)];
                for(size_t k=0; k < N; k++){
                    c[N*n+k] -= elemf * c[N*m+k];
                    unit[N*n+k] -= elemf * unit[N*m+k];
                }
            }
        }
    }
    for(size_t m=0; m < N; m++){ 
        elemf = c[m*(N+1)];
        for(size_t n=0; n < N; n++){
            c[N*m+n] /= elemf;
            unit[N*m+n] /= elemf;
        }
    }
    return unitm;
}

mp_obj_t linalg_inv(mp_obj_t o_in) {
    ndarray_obj_t *o = MP_OBJ_TO_PTR(o_in);
    if(!MP_OBJ_IS_TYPE(o_in, &ulab_ndarray_type)) {
        mp_raise_TypeError("only ndarray objects can be inverted");
    }
    if(o->m != o->n) {
        mp_raise_ValueError("only square matrices can be inverted");
    }
    ndarray_obj_t *inverted = invert_matrix(o->data, o->m);
    return MP_OBJ_FROM_PTR(inverted);
}

mp_obj_t linalg_multiply(mp_obj_t _m1, mp_obj_t _m2) {
    ndarray_obj_t *m1 = MP_OBJ_TO_PTR(_m1);
    ndarray_obj_t *m2 = MP_OBJ_TO_PTR(_m2);    
    if(m1->n != m2->m) {
        mp_raise_ValueError("matrix dimensions do not match");
    }
    ndarray_obj_t *out = create_new_ndarray(m1->m, m2->n, NDARRAY_FLOAT);
    float sum;
    for(size_t i=0; i < m1->n; i++) {
        for(size_t j=0; j < m2->m; j++) {
            for(size_t k=0; k < m1->m; k++) {
                sum += m1->data->items[i*m1->n+k]*m2->data->items[k*m2->n+j];
            }
            mp_set_out->data->items
        }
    }
    return MP_OBJ_FROM_PTR(out);    
}

written 4818 bytes to linalg.c


# Vectorising mathematical operations

## General comments

The following module implements the common mathematical functions for scalars, ndarrays (linear or matrix), and iterables. If the input argument is a scalar, a scalar is returned (i.e., for such arguments, these functions are identical to the functions in the `math` module), while for ndarrays, and iterables, the return value is an ndarray of type `float`. 

In [797]:
%%micropython

import ulab

a = ulab.ndarray([1, 2, 3])
print(ulab.exp(range(5)))
print(ulab.exp(2.0))
print(ulab.exp(a))

ndarray([1.0, 2.718281745910645, 7.389056205749512, 20.08553695678711, 54.59814834594727])

7.38905609893065
ndarray([2.718281745910645, 7.389056205749512, 20.08553695678711])





Note that ndarrays are linear arrays in memory, even if the `shape` of the ndarray is a matrix. This means that we can treat both cases in a *single* loop.

Since `ndarray`s are iterable, we could treat `ndarray`s, `list`s, `tuples`, and `range`s on the same footing. However, that would mean extra trips to a lot of functions, therefore, reading out the values of the `ndarray` directly is probably significantly faster. 

## vectorise.h

In [190]:
%%ccode vectorise.h

#ifndef _VECTORISE_
#define _VECTORISE_

#include "ndarray.h"

mp_obj_t vectorise_acos(mp_obj_t );
mp_obj_t vectorise_acosh(mp_obj_t );
mp_obj_t vectorise_asin(mp_obj_t );
mp_obj_t vectorise_asinh(mp_obj_t );
mp_obj_t vectorise_atan(mp_obj_t );
mp_obj_t vectorise_atanh(mp_obj_t );
mp_obj_t vectorise_ceil(mp_obj_t );
mp_obj_t vectorise_cos(mp_obj_t );
mp_obj_t vectorise_erf(mp_obj_t );
mp_obj_t vectorise_erfc(mp_obj_t );
mp_obj_t vectorise_exp(mp_obj_t );
mp_obj_t vectorise_expm1(mp_obj_t );
mp_obj_t vectorise_floor(mp_obj_t );
mp_obj_t vectorise_gamma(mp_obj_t );
mp_obj_t vectorise_lgamma(mp_obj_t );
mp_obj_t vectorise_log(mp_obj_t );
mp_obj_t vectorise_log10(mp_obj_t );
mp_obj_t vectorise_log2(mp_obj_t );
mp_obj_t vectorise_sin(mp_obj_t );
mp_obj_t vectorise_sinh(mp_obj_t );
mp_obj_t vectorise_sqrt(mp_obj_t );
mp_obj_t vectorise_tan(mp_obj_t );
mp_obj_t vectorise_tanh(mp_obj_t );

#endif

written 1082 bytes to vectorise.h


## vectorise.c

In [798]:
%%ccode vectorise.c

#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include <complex.h>
#include "py/runtime.h"
#include "py/binary.h"
#include "py/obj.h"
#include "py/objarray.h"
#include "vectorise.h"

#ifndef MP_PI
#define MP_PI MICROPY_FLOAT_CONST(3.14159265358979323846)
#endif

mp_obj_t vectorise_generic_vector(mp_obj_t o_in, mp_float_t (*f)(mp_float_t)) {
    // Return a single value, if o_in is not iterable
    if(mp_obj_is_float(o_in) || mp_obj_is_integer(o_in)) {
            return mp_obj_new_float(f(mp_obj_get_float(o_in)));
    }
    mp_float_t x;
    if(MP_OBJ_IS_TYPE(o_in, &ulab_ndarray_type)) {
        ndarray_obj_t *o = MP_OBJ_TO_PTR(o_in);
        ndarray_obj_t *out = create_new_ndarray(o->m, o->n, NDARRAY_FLOAT);
        float *datain = (float *)o->data->items;
        float *dataout = (float *)out->data->items;
        for(size_t i=0; i < o->data->len; i++) {
            dataout[i] = f(datain[i]);
        }
        return MP_OBJ_FROM_PTR(out);
    } else if(MP_OBJ_IS_TYPE(o_in, &mp_type_tuple) || MP_OBJ_IS_TYPE(o_in, &mp_type_list) || 
        MP_OBJ_IS_TYPE(o_in, &mp_type_range)) {
            mp_obj_array_t *o = MP_OBJ_TO_PTR(o_in);
            ndarray_obj_t *out = create_new_ndarray(1, o->len, NDARRAY_FLOAT);
            float *dataout = (float *)out->data->items;
            mp_obj_iter_buf_t iter_buf;
            mp_obj_t item, iterable = mp_getiter(o_in, &iter_buf);
            size_t i=0;
            while ((item = mp_iternext(iterable)) != MP_OBJ_STOP_ITERATION) {
                x = mp_obj_get_float(item);
                dataout[i++] = f(x);
            }
        return MP_OBJ_FROM_PTR(out);
    }
    return mp_const_none;
}


#define MATH_FUN_1(py_name, c_name) \
    mp_obj_t vectorise_ ## py_name(mp_obj_t x_obj) { \
        return vectorise_generic_vector(x_obj, MICROPY_FLOAT_C_FUN(c_name)); \
    }

// _degrees won't compile for the unix port
/*
mp_float_t _degreesf(mp_float_t x) {
    return(180*x/MP_PI);
}

MATH_FUN_1(degrees, _degrees);

// _radians won't compile for the unix port
mp_float_t _radiansf(mp_float_t x) {
    return(MP_PI*x/180.0);
}

MATH_FUN_1(radians, _radians);

STATIC mp_float_t _fabsf(mp_float_t x) {
    return fabsf(x);
}

MATH_FUN_1(fabs, _fabs);
*/
MATH_FUN_1(acos, acos);
MATH_FUN_1(acosh, acosh);
MATH_FUN_1(asin, asin);
MATH_FUN_1(asinh, asinh);
MATH_FUN_1(atan, atan);	
MATH_FUN_1(atanh, atanh);
MATH_FUN_1(ceil, ceil);
MATH_FUN_1(cos, cos);
MATH_FUN_1(erf, erf);
MATH_FUN_1(erfc, erfc);
MATH_FUN_1(exp, exp);
MATH_FUN_1(expm1, expm1);
MATH_FUN_1(floor, floor);
MATH_FUN_1(gamma, tgamma);
MATH_FUN_1(lgamma, lgamma);
MATH_FUN_1(log, log);
MATH_FUN_1(log10, log10);
MATH_FUN_1(log2, log2);
MATH_FUN_1(sin, sin);
MATH_FUN_1(sinh, sinh);
MATH_FUN_1(sqrt, sqrt);
MATH_FUN_1(tan, tan);
MATH_FUN_1(tanh, tanh);

written 2972 bytes to vectorise.c


# Polynomial fits

In [800]:
%%micropython

import ulab

p = [1, 2, 3, 4]
x = [0, 1, 2, 3, 4]

print(ulab.polyval(p, x))
a = ulab.ndarray(x)
print(a)
print(ulab.polyval(p, a))

ndarray([4.0, 10.0, 26.0, 58.00000000000001, 112.0])

ndarray([0.0, 1.0, 2.0, 3.0, 4.0])

ndarray([4.0, 10.0, 26.0, 58.00000000000001, 112.0])





## poly.h

In [754]:
%%ccode poly.h

#ifndef _POLY_
#define _POLY_

mp_obj_t poly_polyval(mp_obj_t , mp_obj_t );

#endif

written 264 bytes to poly.h


## poly.c

In [819]:
%%ccode poly.c

#include "py/runtime.h"
#include "py/objarray.h"
#include "ndarray.h"
#include "poly.h"

mp_obj_t poly_polyval(mp_obj_t o_p, mp_obj_t o_x) {
    // TODO: return immediately, if o_p is not an iterable
    size_t m, n;
    if(MP_OBJ_IS_TYPE(o_x, &ulab_ndarray_type)) {
        ndarray_obj_t *ndx = MP_OBJ_TO_PTR(o_x);
        m = ndx->m;
        n = ndx->n;
    } else {
        mp_obj_array_t *ix = MP_OBJ_TO_PTR(o_x);
        m = 1;
        n = ix->len;
    }
    // polynomials are going to be of float, except, when both 
    // the coefficients and the independent variable are integers
    ndarray_obj_t *out = create_new_ndarray(m, n, NDARRAY_FLOAT);
    mp_obj_iter_buf_t x_buf;
    mp_obj_t x_item, x_iterable = mp_getiter(o_x, &x_buf);

    mp_obj_iter_buf_t p_buf;
    mp_obj_t p_item, p_iterable;

    size_t i = 0;
    mp_float_t x, y;
    float *outf = (float *)out->data->items;
    while ((x_item = mp_iternext(x_iterable)) != MP_OBJ_STOP_ITERATION) {
        x = mp_obj_get_float(x_item);
        p_iterable = mp_getiter(o_p, &p_buf);
        p_item = mp_iternext(p_iterable);
        y = mp_obj_get_float(p_item);
        while((p_item = mp_iternext(p_iterable)) != MP_OBJ_STOP_ITERATION) {
            y *= x;
            y += mp_obj_get_float(p_item);
        }
        outf[i++] = y;
    }
    return MP_OBJ_FROM_PTR(out);
}

written 1524 bytes to poly.c


# Fast Fourier transform

In [830]:
%%micropython

import ulab

print(ulab.fft([1, 2, 3, 4]*2, [0]*8))

(ndarray([164982.0, -14252.443359375, -142.0067901611328, 14252.4453125, -164690.0, 14252.443359375, -141.9932098388672, -14252.4453125])
, ndarray([0.0, 15031.0322265625, -155236.0, 15103.0283203125, 0.0, -15103.0322265625, 155236.0, -15031.0283203125])
)




## fft.h

In [807]:
%%ccode fft.h

#ifndef _FFT_
#define _FFT_

#ifndef MP_PI
#define MP_PI MICROPY_FLOAT_CONST(3.14159265358979323846)
#endif

#include <complex.h>
typedef float complex complex_t;

mp_obj_t fft_fft(mp_obj_t, mp_obj_t);
#endif

written 389 bytes to fft.h


## fft.c

In [823]:
%%ccode fft.c

#include <math.h>
#include <stdlib.h>
#include "py/runtime.h"
#include "ndarray.h"
#include "fft.h"
    
STATIC void fft_kernel(complex_t *buffer, complex_t *out, int n, int step) {
    // this is the actual FFT kernel that we call recursively
    if (step < n) {
        // split the input buffer in two
        fft_kernel(out, buffer, n, step*2);
        fft_kernel(out+step, buffer+step, n, step*2);

        for (int i=0; i<n; i+=2*step) {
            complex_t twiddle = (cosf(MP_PI*i/n)-I*sinf(MP_PI*i/n)) * out[i+step];
            buffer[i/2] = out[i] + twiddle * out[i+step];
            buffer[(i+n)/2] = out[i] - twiddle * out[i+step];
        }
    }
}

STATIC void fft_real_kernel(float *re, float *im, float *out_re, float *out_im, int n, int step) {
    // this is the actual FFT kernel that we call recursively
    if (step < n) {
        // split the input buffer in two
        fft_real_kernel(out_re, out_im, re, im, n, step*2);
        fft_real_kernel(out_re+step, out_im+step, re+step, im+step, n, step*2);

        float twiddle_re, twiddle_im;
        for (int i=0; i<n; i+=2*step) {
//            complex_t twiddle = (cosf(MP_PI*i/n)-I*sinf(MP_PI*i/n)) * out[i+step];
            twiddle_re = cosf(MP_PI*i/n) * out_re[i+step] + sinf(MP_PI*i/n) * out_im[i+step];
            twiddle_im = cosf(MP_PI*i/n) * out_im[i+step] - sinf(MP_PI*i/n) * out_re[i+step];
//            buffer[i/2] = out[i] + twiddle * out[i+step];
            re[i/2] = out_re[i] + twiddle_re * out_re[i+step] - twiddle_im * out_im[i+step];
            im[i/2] = out_im[i] + twiddle_re * out_im[i+step] - twiddle_im * out_re[i+step];
            buffer[(i+n)/2] = out[i] - twiddle * out[i+step];
            re[(i+n)/2] = out_re[i] - twiddle_re * out_re[i+step] + twiddle_im * out_im[i+step];
            im[(i+n)/2] = out_im[i] - twiddle_re * out_im[i+step] - twiddle_im * out_re[i+step];
        }
    }
}

mp_obj_t fft_fft(mp_obj_t real, mp_obj_t imag) {
    // TODO: return the absolute value, if keyword argument is specified
    // TODO: let go of the complex type, for the assignment of values is through reals and imags, anyway
    // Check if input is of length of power of 2
    mp_obj_array_t *re = MP_OBJ_TO_PTR(real);
    if ((re->len & (re->len-1)) != 0) {
        mp_raise_ValueError("input array length must be power of 2");
    }
    mp_obj_array_t *im = MP_OBJ_TO_PTR(imag);
    if (re->len != im->len) {
        mp_raise_ValueError("real and imaginary parts must be of equal length");
    }

    complex_t *buffer, *out;
    buffer = (complex_t *)malloc(sizeof(complex_t)*re->len);
    out = (complex_t *)malloc(sizeof(complex_t)*re->len);
    size_t i=0;

    if(MP_OBJ_IS_TYPE(re, &ulab_ndarray_type) && MP_OBJ_IS_TYPE(im, &ulab_ndarray_type)) {
        ndarray_obj_t *_re = MP_OBJ_TO_PTR(real);
        ndarray_obj_t *_im = MP_OBJ_TO_PTR(imag);
        for(size_t j=0; j < re->len; j++) {
            buffer[i] = out[i] = ndarray_get_float_value(_re->data->items, _re->data->typecode, j) + 
            I*ndarray_get_float_value(_im->data->items, _im->data->typecode, j);
        }
    } else {
        mp_obj_iter_buf_t re_buf;
        mp_obj_t re_item, re_iterable = mp_getiter(real, &re_buf);

        mp_obj_iter_buf_t im_buf;
        mp_obj_t im_item, im_iterable = mp_getiter(imag, &im_buf);

        while ((re_item = mp_iternext(re_iterable)) != MP_OBJ_STOP_ITERATION) {
            im_item = mp_iternext(im_iterable);
            buffer[i] = out[i] = mp_obj_get_float(re_item) + I*mp_obj_get_float(im_item);
            i++;
        }
    }
    fft_kernel(buffer, out, re->len, 1);

    complex_t z;
    if(1) {
        ndarray_obj_t *re_out = create_new_ndarray(1, re->len, NDARRAY_FLOAT);
        ndarray_obj_t *im_out = create_new_ndarray(1, im->len, NDARRAY_FLOAT);
        float *_re_out = (float *)re_out->data->items;
        float *_im_out = (float *)im_out->data->items;        
        for(i=0; i<re->len; i++) {
            z = buffer[i];
            _re_out[i] = creal(z);
            _im_out[i] = cimag(z);
        }
        free(buffer);
        mp_obj_t tuple[2];
        tuple[0] = re_out;
        tuple[1] = im_out;
        return mp_obj_new_tuple(2, tuple);
    } 
}

written 3153 bytes to fft.c


# Numerical

## General comments

This section contains miscellaneous functions that did not fit in the other submodules. These include `linspace`, `logspace`, `min/max`, `argmin/argmax`, `sum`, `mean`, `std`, and `clip`. These latter functions work with iterables, or ndarrays. When the ndarray is two-dimensional, an `axis` keyword can be supplied, in which case, the function returns a vector, otherwise a scalar.

Since the return values of `mean`, and `std` are most probably floats, these functions return ndarrays of type float, while `min/max` and `clip` do not change the type, and `argmin/argmax` return `uint8`, if the values are smaller than 255, otherwise, `uint16`.

## Examples

In [249]:
%%micropython

import ulab

print(ulab.linspace(0, 10, 11))
print(ulab.sum([1, 2, 3]))

ndarray([0.0, 1.0, 2.0, ..., 8.0, 9.0, 10.0])

6.0




## numerical.h

In [16]:
%%ccode numerical.h

#ifndef _NUMERICAL_
#define _NUMERICAL_

#include "ndarray.h"

mp_obj_t numerical_linspace(mp_obj_t , mp_obj_t , mp_obj_t );
mp_obj_t numerical_sum(size_t , const mp_obj_t *, mp_map_t *);
mp_obj_t numerical_mean(size_t , const mp_obj_t *, mp_map_t *);
mp_obj_t numerical_std(size_t , const mp_obj_t *, mp_map_t *);
mp_obj_t numerical_min(size_t , const mp_obj_t *, mp_map_t *);
mp_obj_t numerical_max(size_t , const mp_obj_t *, mp_map_t *);
mp_obj_t numerical_argmin(size_t , const mp_obj_t *, mp_map_t *);
mp_obj_t numerical_argmax(size_t , const mp_obj_t *, mp_map_t *);
mp_obj_t numerical_roll(size_t , const mp_obj_t *, mp_map_t *);

#endif

written 825 bytes to numerical.h


## numerical.c

### Parsing of arguments

Since most of these functions operate on matrices along an axis, it might make sense to factor out the parsing of arguments and keyword arguments. The void function `numerical_parse_args` fills in the pointer for the matrix/array, and the axis.

In [694]:
%%ccode numerical.c

#include <math.h>
#include "py/obj.h"
#include "py/runtime.h"
#include "py/builtin.h"
#include "numerical.h"

enum NUMERICAL_FUNCTION_TYPE {
    NUMERICAL_MIN,
    NUMERICAL_MAX,
    NUMERICAL_ARGMIN,
    NUMERICAL_ARGMAX,
    NUMERICAL_SUM,
    NUMERICAL_MEAN,
    NUMERICAL_STD,
    NUMERICAL_ROLL,
};

mp_obj_t numerical_linspace(mp_obj_t _start, mp_obj_t _stop, mp_obj_t _len) {
    // TODO: accept keyword argument endpoint=True
    mp_int_t len = mp_obj_get_int_truncated(_len);
    if(len < 2) {
        mp_raise_ValueError("number of points must be at least 2");
    }
    mp_float_t value, step;
    value = mp_obj_get_float(_start);
    step = (mp_obj_get_float(_stop)-value)/(len-1);
    ndarray_obj_t *nd_array = create_new_ndarray(1, len, NDARRAY_FLOAT);
    for(size_t i=0; i < len; i++, value += step) {
        mp_binary_set_val_array('f', nd_array->data->items, i, mp_obj_new_float(value));
    }
    return MP_OBJ_FROM_PTR(nd_array);
}

mp_obj_t numerical_sum_mean_std_array(mp_obj_t oin, uint8_t optype) {
    mp_float_t value, sum = 0.0, sq_sum = 0.0;
    mp_obj_iter_buf_t iter_buf;
    mp_obj_t item, iterable = mp_getiter(oin, &iter_buf);
    mp_int_t len = mp_obj_get_int(mp_obj_len(oin));
    while ((item = mp_iternext(iterable)) != MP_OBJ_STOP_ITERATION) {
        value = mp_obj_get_float(item);
        sum += value;
        if(optype == NUMERICAL_STD) {
            sq_sum += value*value;
        }
    }
    if(optype ==  NUMERICAL_SUM) {
        return mp_obj_new_float(sum);
    } else if(optype == NUMERICAL_MEAN) {
        return mp_obj_new_float(sum/len);
    } else {
        sum /= len; // this is now the mean!
        return mp_obj_new_float(sqrtf((sq_sum/len-sum*sum)));
    }
}

STATIC mp_obj_t numerical_argmin_argmax_array(mp_obj_t o_in, mp_uint_t op, uint8_t type) {
    size_t idx = 0, best_idx = 0;
    mp_obj_iter_buf_t iter_buf;
    mp_obj_t iterable = mp_getiter(o_in, &iter_buf);
    mp_obj_t best_obj = MP_OBJ_NULL;
    mp_obj_t item;
    while ((item = mp_iternext(iterable)) != MP_OBJ_STOP_ITERATION) {
        if ((best_obj == MP_OBJ_NULL) || (mp_binary_op(op, item, best_obj) == mp_const_true)) {
            best_obj = item;
            best_idx = idx;
        }
        idx++;
    }
    if((type == NUMERICAL_ARGMIN) || (type == NUMERICAL_ARGMAX)) {
        return MP_OBJ_NEW_SMALL_INT(best_idx);
    } else {
        return best_obj;
    }
}

STATIC size_t numerical_argmin_argmax_single_line(void *data, size_t start, size_t stop, 
                                                  size_t stride, uint8_t typecode, uint8_t optype) {
    size_t best_idx = start;
    mp_float_t value, best_value = ndarray_get_float_value(data, typecode, start);
    
    for(size_t i=start; i < stop; i+=stride) {
        value = ndarray_get_float_value(data, typecode, i);
        if((optype == NUMERICAL_MIN) || (optype == NUMERICAL_ARGMIN)) {
            if(best_value > value) {
                best_value = value;
                best_idx = i;
            }
        } else if((optype == NUMERICAL_MAX) || (optype == NUMERICAL_ARGMAX)) {
            if(best_value < value) {
                best_value = value;
                best_idx = i;
            }
        }
    }
    return best_idx;
}

STATIC mp_obj_t numerical_argmin_argmax_matrix(mp_obj_t oin, mp_obj_t axis, uint8_t optype) {
    ndarray_obj_t *in = MP_OBJ_TO_PTR(oin);
    size_t best_idx;
    if((axis == mp_const_none) || (in->m == 1) || (in->n == 1)) { 
        // return the value for the flattened array
        best_idx = numerical_argmin_argmax_single_line(in->data->items, 0, 
                                                      in->data->len, 1, in->data->typecode, optype);
        if((optype == NUMERICAL_ARGMIN) || (optype == NUMERICAL_ARGMAX)) {
            return MP_OBJ_NEW_SMALL_INT(best_idx);
        } else {
            // TODO: do we have to do type conversion here, depending on the type of the input array?
            return mp_obj_new_float(ndarray_get_float_value(in->data->items, in->data->typecode, best_idx));
        }
    } else {
        uint8_t _axis = mp_obj_get_int(axis);
        size_t m = (_axis == 0) ? 1 : in->m;
        size_t n = (_axis == 0) ? in->n : 1;
        size_t len = in->data->len;
        // TODO: pass in->data->typcode to create_new_ndarray
        ndarray_obj_t *out = create_new_ndarray(m, n, NDARRAY_FLOAT);

        // TODO: these two cases could probably be combined in a more elegant fashion...
        if(_axis == 0) { // vertical
            for(size_t i=0; i < n; i++) {
                best_idx = numerical_argmin_argmax_single_line(in->data->items, i, len, 
                                                               n, in->data->typecode, optype);
                if((optype == NUMERICAL_ARGMIN) || (optype == NUMERICAL_ARGMAX)) {
                    ((float_t *)out->data->items)[i] = (float)best_idx;
                } else {
                    ((float_t *)out->data->items)[i] = ndarray_get_float_value(in->data->items, in->data->typecode, best_idx);
                }
            }
        } else { // horizontal
            for(size_t i=0; i < m; i++) {
                best_idx = numerical_argmin_argmax_single_line(in->data->items, i*in->n, 
                                                               (i+1)*in->n, 1, in->data->typecode, optype);
                if((optype == NUMERICAL_ARGMIN) || (optype == NUMERICAL_ARGMAX)) {
                    ((float_t *)out->data->items)[i] = (float)best_idx;
                } else {
                    ((float_t *)out->data->items)[i] = ndarray_get_float_value(in->data->items, in->data->typecode, best_idx);
                }

            }
        }
    return MP_OBJ_FROM_PTR(out);
    }
    return mp_const_none;
}

STATIC mp_float_t numerical_sum_mean_std_single_line(void *data, size_t start, size_t stop, 
                                                  size_t stride, uint8_t typecode, uint8_t optype) {
    
    mp_float_t sum = 0.0, sq_sum = 0.0, value;
    size_t len = 0;
    for(size_t i=start; i < stop; i+=stride, len++) {
        value = ndarray_get_float_value(data, typecode, i);        
        sum += value;
        if(optype == NUMERICAL_STD) {
            sq_sum += value*value;
        }
    }
    if(len == 0) {
        mp_raise_ValueError("data length is 0!");
    }
    if(optype ==  NUMERICAL_SUM) {
        return sum;
    } else if(optype == NUMERICAL_MEAN) {
        return sum/len;
    } else {
        sum /= len; // this is now the mean!
        return sqrtf((sq_sum/len-sum*sum));
    }
}

STATIC mp_obj_t numerical_sum_mean_std_matrix(mp_obj_t oin, mp_obj_t axis, uint8_t optype) {
    ndarray_obj_t *in = MP_OBJ_TO_PTR(oin);
    if((axis == mp_const_none) || (in->m == 1) || (in->n == 1)) { 
        // return the value for the flattened array
        return mp_obj_new_float(numerical_sum_mean_std_single_line(in->data->items, 0, 
                                                      in->data->len, 1, in->data->typecode, optype));
    } else {
        uint8_t _axis = mp_obj_get_int(axis);
        size_t m = (_axis == 0) ? 1 : in->m;
        size_t n = (_axis == 0) ? in->n : 1;
        size_t len = in->data->len;
        mp_float_t sms;
        // TODO: pass in->data->typcode to create_new_ndarray
        ndarray_obj_t *out = create_new_ndarray(m, n, NDARRAY_FLOAT);

        // TODO: these two cases could probably be combined in a more elegant fashion...
        if(_axis == 0) { // vertical
            for(size_t i=0; i < n; i++) {
                sms = numerical_sum_mean_std_single_line(in->data->items, i, len, 
                                                               n, in->data->typecode, optype);
                ((float_t *)out->data->items)[i] = sms;
            }
        } else { // horizontal
            for(size_t i=0; i < m; i++) {
                sms = numerical_sum_mean_std_single_line(in->data->items, i*in->n, 
                                                               (i+1)*in->n, 1, in->data->typecode, optype);
                ((float_t *)out->data->items)[i] = sms;
            }
        }
    return MP_OBJ_FROM_PTR(out);
    }
}

STATIC mp_obj_t numerical_function(size_t n_args, const mp_obj_t *pos_args, mp_map_t *kw_args, uint8_t type) {
    static const mp_arg_t allowed_args[] = {
        { MP_QSTR_oin, MP_ARG_REQUIRED | MP_ARG_OBJ, {.u_rom_obj = MP_ROM_PTR(&mp_const_none_obj)} } ,
        { MP_QSTR_axis, MP_ARG_KW_ONLY | MP_ARG_OBJ, {.u_rom_obj = MP_ROM_PTR(&mp_const_none_obj)} },
    };

    mp_arg_val_t args[MP_ARRAY_SIZE(allowed_args)];
    mp_arg_parse_all(1, pos_args, kw_args, MP_ARRAY_SIZE(allowed_args), allowed_args, args);
    
    mp_obj_t oin = args[0].u_obj;
    mp_obj_t axis = args[1].u_obj;
    if((axis != mp_const_none) && (mp_obj_get_int(axis) != 0) && (mp_obj_get_int(axis) != 1)) {
        // this seems to pass with False, and True...
        mp_raise_ValueError("axis must be None, 0, or 1");
    }
    
    if(MP_OBJ_IS_TYPE(oin, &mp_type_tuple) || MP_OBJ_IS_TYPE(oin, &mp_type_list) || 
        MP_OBJ_IS_TYPE(oin, &mp_type_range)) {
        switch(type) {
            case NUMERICAL_MIN:
            case NUMERICAL_ARGMIN:
                return numerical_argmin_argmax_array(oin, MP_BINARY_OP_LESS, type);
            case NUMERICAL_MAX:
            case NUMERICAL_ARGMAX:
                return numerical_argmin_argmax_array(oin, MP_BINARY_OP_MORE, type);
            case NUMERICAL_SUM:
            case NUMERICAL_MEAN:
            case NUMERICAL_STD:
                return numerical_sum_mean_std_array(oin, type);
            default: // we should never reach this point, but whatever
                return mp_const_none;
        }
    } else if(MP_OBJ_IS_TYPE(oin, &ulab_ndarray_type)) {
        switch(type) {
            case NUMERICAL_MIN:
            case NUMERICAL_MAX:
            case NUMERICAL_ARGMIN:
            case NUMERICAL_ARGMAX:
                return numerical_argmin_argmax_matrix(oin, axis, type);
            case NUMERICAL_SUM:
            case NUMERICAL_MEAN:
            case NUMERICAL_STD:
                return numerical_sum_mean_std_matrix(oin, axis, type);            
            default:
                mp_raise_NotImplementedError("operation is not implemented on ndarrays");
        }
    } else {
        mp_raise_TypeError("input must be tuple, list, range, or ndarray");
    }
    return mp_const_none;
}

mp_obj_t numerical_min(size_t n_args, const mp_obj_t *pos_args, mp_map_t *kw_args) {
    return numerical_function(n_args, pos_args, kw_args, NUMERICAL_MIN);
}

mp_obj_t numerical_max(size_t n_args, const mp_obj_t *pos_args, mp_map_t *kw_args) {
    return numerical_function(n_args, pos_args, kw_args, NUMERICAL_MAX);
}

mp_obj_t numerical_argmin(size_t n_args, const mp_obj_t *pos_args, mp_map_t *kw_args) {
    return numerical_function(n_args, pos_args, kw_args, NUMERICAL_ARGMIN);
}

mp_obj_t numerical_argmax(size_t n_args, const mp_obj_t *pos_args, mp_map_t *kw_args) {
    return numerical_function(n_args, pos_args, kw_args, NUMERICAL_ARGMAX);
}

mp_obj_t numerical_sum(size_t n_args, const mp_obj_t *pos_args, mp_map_t *kw_args) {
    return numerical_function(n_args, pos_args, kw_args, NUMERICAL_SUM);
}

mp_obj_t numerical_mean(size_t n_args, const mp_obj_t *pos_args, mp_map_t *kw_args) {
    return numerical_function(n_args, pos_args, kw_args, NUMERICAL_MEAN);
}

mp_obj_t numerical_std(size_t n_args, const mp_obj_t *pos_args, mp_map_t *kw_args) {
    return numerical_function(n_args, pos_args, kw_args, NUMERICAL_STD);
}

written 12166 bytes to numerical.c


# ulab module

This module simply brings all components together, and does not contain new function definitions.

## ulab.c

In [825]:
%%ccode ulab.c

#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "py/runtime.h"
#include "py/binary.h"
#include "py/obj.h"
#include "py/objarray.h" // this can in the future be dropped

#include "ndarray.h"
#include "linalg.h"
#include "vectorise.h"
#include "poly.h"
#include "fft.h"
#include "numerical.h"

#define ULAB__VERSION 1.0

MP_DEFINE_CONST_FUN_OBJ_1(ndarray_shape_obj, ndarray_shape);
MP_DEFINE_CONST_FUN_OBJ_2(ndarray_size_obj, ndarray_size);
MP_DEFINE_CONST_FUN_OBJ_1(ndarray_rawsize_obj, ndarray_rawsize);

MP_DEFINE_CONST_FUN_OBJ_1(linalg_transpose_obj, linalg_transpose);
MP_DEFINE_CONST_FUN_OBJ_2(linalg_reshape_obj, linalg_reshape);
MP_DEFINE_CONST_FUN_OBJ_1(linalg_inv_obj, linalg_inv);

MP_DEFINE_CONST_FUN_OBJ_1(vectorise_acos_obj, vectorise_acos);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_acosh_obj, vectorise_acosh);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_asin_obj, vectorise_asin);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_asinh_obj, vectorise_asinh);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_atan_obj, vectorise_atan);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_atanh_obj, vectorise_atanh);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_ceil_obj, vectorise_ceil);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_cos_obj, vectorise_cos);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_erf_obj, vectorise_erf);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_erfc_obj, vectorise_erfc);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_exp_obj, vectorise_exp);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_expm1_obj, vectorise_expm1);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_floor_obj, vectorise_floor);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_gamma_obj, vectorise_gamma);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_lgamma_obj, vectorise_lgamma);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_log_obj, vectorise_log);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_log10_obj, vectorise_log10);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_log2_obj, vectorise_log2);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_sin_obj, vectorise_sin);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_sinh_obj, vectorise_sinh);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_sqrt_obj, vectorise_sqrt);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_tan_obj, vectorise_tan);
MP_DEFINE_CONST_FUN_OBJ_1(vectorise_tanh_obj, vectorise_tanh);

MP_DEFINE_CONST_FUN_OBJ_3(numerical_linspace_obj, numerical_linspace);
STATIC MP_DEFINE_CONST_FUN_OBJ_KW(numerical_sum_obj, 1, numerical_sum);
STATIC MP_DEFINE_CONST_FUN_OBJ_KW(numerical_mean_obj, 1, numerical_mean);
STATIC MP_DEFINE_CONST_FUN_OBJ_KW(numerical_std_obj, 1, numerical_std);
STATIC MP_DEFINE_CONST_FUN_OBJ_KW(numerical_min_obj, 1, numerical_min);
STATIC MP_DEFINE_CONST_FUN_OBJ_KW(numerical_max_obj, 1, numerical_max);
STATIC MP_DEFINE_CONST_FUN_OBJ_KW(numerical_argmin_obj, 1, numerical_argmin);
STATIC MP_DEFINE_CONST_FUN_OBJ_KW(numerical_argmax_obj, 1, numerical_argmax);

STATIC MP_DEFINE_CONST_FUN_OBJ_2(poly_polyval_obj, poly_polyval);
STATIC MP_DEFINE_CONST_FUN_OBJ_2(fft_fft_obj, fft_fft);

STATIC const mp_rom_map_elem_t ulab_ndarray_locals_dict_table[] = {
    { MP_ROM_QSTR(MP_QSTR_shape), MP_ROM_PTR(&ndarray_shape_obj) },
    { MP_ROM_QSTR(MP_QSTR_size), MP_ROM_PTR(&ndarray_size_obj) },
    { MP_ROM_QSTR(MP_QSTR_rawsize), MP_ROM_PTR(&ndarray_rawsize_obj) },
    { MP_ROM_QSTR(MP_QSTR_transpose), MP_ROM_PTR(&linalg_transpose_obj) },
    { MP_ROM_QSTR(MP_QSTR_reshape), MP_ROM_PTR(&linalg_reshape_obj) },
//    { MP_ROM_QSTR(MP_QSTR_get), MP_ROM_PTR(&ulab_ndarray_get_obj) },
//    { MP_ROM_QSTR(MP_QSTR_dot), MP_ROM_PTR(&ulab_ndarray_dot_obj) },    
    // class constants
    { MP_ROM_QSTR(MP_QSTR_uint8), MP_ROM_INT(NDARRAY_UINT8) },
    { MP_ROM_QSTR(MP_QSTR_int8), MP_ROM_INT(NDARRAY_INT8) },
    { MP_ROM_QSTR(MP_QSTR_uint16), MP_ROM_INT(NDARRAY_UINT16) },
    { MP_ROM_QSTR(MP_QSTR_int16), MP_ROM_INT(NDARRAY_INT16) },
    { MP_ROM_QSTR(MP_QSTR_float), MP_ROM_INT(NDARRAY_FLOAT) },
};

STATIC MP_DEFINE_CONST_DICT(ulab_ndarray_locals_dict, ulab_ndarray_locals_dict_table);

const mp_obj_type_t ulab_ndarray_type = {
    { &mp_type_type },
    .name = MP_QSTR_ndarray,
    .print = ndarray_print,
    .make_new = ndarray_make_new,
    .subscr = ndarray_subscr,
    .getiter = ndarray_getiter,
//    .unary_op = ndarray_unary_op,
//    .binary_op = ndarray_binary_op,
    .locals_dict = (mp_obj_dict_t*)&ulab_ndarray_locals_dict,
};

STATIC const mp_map_elem_t ulab_globals_table[] = {
    { MP_OBJ_NEW_QSTR(MP_QSTR___name__), MP_OBJ_NEW_QSTR(MP_QSTR_ulab) },
    { MP_OBJ_NEW_QSTR(MP_QSTR_ndarray), (mp_obj_t)&ulab_ndarray_type },
    { MP_OBJ_NEW_QSTR(MP_QSTR_inv), (mp_obj_t)&linalg_inv_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_acos), (mp_obj_t)&vectorise_acos_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_acosh), (mp_obj_t)&vectorise_acosh_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_asin), (mp_obj_t)&vectorise_asin_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_asinh), (mp_obj_t)&vectorise_asinh_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_atan), (mp_obj_t)&vectorise_atan_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_atanh), (mp_obj_t)&vectorise_atanh_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_ceil), (mp_obj_t)&vectorise_ceil_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_cos), (mp_obj_t)&vectorise_cos_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_erf), (mp_obj_t)&vectorise_erf_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_erfc), (mp_obj_t)&vectorise_erfc_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_exp), (mp_obj_t)&vectorise_exp_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_expm1), (mp_obj_t)&vectorise_expm1_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_floor), (mp_obj_t)&vectorise_floor_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_gamma), (mp_obj_t)&vectorise_gamma_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_lgamma), (mp_obj_t)&vectorise_lgamma_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_log), (mp_obj_t)&vectorise_log_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_log10), (mp_obj_t)&vectorise_log10_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_log2), (mp_obj_t)&vectorise_log2_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_sin), (mp_obj_t)&vectorise_sin_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_sinh), (mp_obj_t)&vectorise_sinh_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_sqrt), (mp_obj_t)&vectorise_sqrt_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_tan), (mp_obj_t)&vectorise_tan_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_tanh), (mp_obj_t)&vectorise_tanh_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_linspace), (mp_obj_t)&numerical_linspace_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_sum), (mp_obj_t)&numerical_sum_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_mean), (mp_obj_t)&numerical_mean_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_std), (mp_obj_t)&numerical_std_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_min), (mp_obj_t)&numerical_min_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_max), (mp_obj_t)&numerical_max_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_argmin), (mp_obj_t)&numerical_argmin_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_argmax), (mp_obj_t)&numerical_argmax_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_polyval), (mp_obj_t)&poly_polyval_obj },
    { MP_OBJ_NEW_QSTR(MP_QSTR_fft), (mp_obj_t)&fft_fft_obj },
};

STATIC MP_DEFINE_CONST_DICT (
    mp_module_ulab_globals,
    ulab_globals_table
);

const mp_obj_module_t ulab_user_cmodule = {
    .base = { &mp_type_module },
    .globals = (mp_obj_dict_t*)&mp_module_ulab_globals,
};


MP_REGISTER_MODULE(MP_QSTR_ulab, ulab_user_cmodule, MODULE_ULAB_ENABLED);

written 7323 bytes to ulab.c


## makefile

In [805]:
%%writefile ../../../usermod/ulab/micropython.mk

USERMODULES_DIR := $(USERMOD_DIR)

# Add all C files to SRC_USERMOD.
SRC_USERMOD += $(USERMODULES_DIR)/ndarray.c
SRC_USERMOD += $(USERMODULES_DIR)/linalg.c
SRC_USERMOD += $(USERMODULES_DIR)/vectorise.c
SRC_USERMOD += $(USERMODULES_DIR)/poly.c
SRC_USERMOD += $(USERMODULES_DIR)/fft.c
SRC_USERMOD += $(USERMODULES_DIR)/numerical.c
SRC_USERMOD += $(USERMODULES_DIR)/ulab.c

# We can add our module folder to include paths if needed
# This is not actually needed in this example.
CFLAGS_USERMOD += -I$(USERMODULES_DIR)

Overwriting ../../../usermod/ulab/micropython.mk


## make

In [164]:
!make USER_C_MODULES=../../../ulab all

Use make V=1 or set BUILD_VERBOSE in your environment to increase build verbosity.
Including User C Module from ../../../ulab/code
GEN build/genhdr/moduledefs.h
GEN build/genhdr/qstr.i.last
GEN build/genhdr/qstr.split
GEN build/genhdr/qstrdefs.collected.h
QSTR not updated
CC ../../py/objmodule.c
CC ../../../ulab/code/ndarray.c
LINK micropython
   text	   data	    bss	    dec	    hex	filename
   2085	   6862	      0	   8947	   22f3	build/build/frozen_mpy.o
      2	      0	      0	      2	      2	build/build/frozen.o
 455198	  56992	   2104	 514294	  7d8f6	micropython


In [66]:
%%micropython

import ulab

a = ulab.ndarray([1, 2, 3, 4], dtype='float')
print(ulab.max(a))
print(a[1:3])

4.0
ndarray([2.0, 3.0], dtype='float')





In [166]:
%%micropython

import ulab

a = ulab.ndarray([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]], dtype='uint8')

for i in a:
    print(i)

ndarray([1, 2, 3, 4], dtype='uint8')

ndarray([5, 6, 7, 8], dtype='uint8')

ndarray([9, 10, 11, 12], dtype='uint8')





In [691]:
%%micropython

import ulab
a = ulab.ndarray([11, 2, 3, 4])
print(ulab.max(a))
# print(ulab.min([1, 2, 3, 4]))
print(ulab.mean(ulab.ndarray([[1, 2, 3], [5, 6, 7]]), axis=1))
print(ulab.mean(ulab.ndarray([[1, 2, 3], [5, 6, 7]])))
print(ulab.sum(ulab.ndarray([[1, 2, 3], [5, 6, 7]])))
print(ulab.std(ulab.ndarray([[1, 2, 3], [5, 6, 7]])))

11.0
ndarray([2.0, 6.0])

4.0
24.0
2.160246849060059




In [692]:
%%micropython

import ulab

print(ulab.acos([0.1, 2.2]))
print(ulab.exp([0.1, 2.2]))
# print(ulab.sum(ulab.ndarray([[1, 2, 3], [5, 6, 7]]), axis=2))
print(ulab.mean(range(10)))
# print(ulab.mean(ulab.ndarray([[1, 2, 3], [5, 6, 7]]), axis=2))
print(ulab.std(range(100)))

ndarray([1.47062885761261, nan])

ndarray([1.105170965194702, 9.02501392364502])

4.5
28.86606979370118




In [438]:
std(range(100))

28.86607004772212

In [1]:
%%micropython

import ulab
a = ulab.ndarray([[1, 2, 3], [5, 6, 7]])
print(a)
print(a.shape())
print(a.size(1))
a.transpose()
print(a.shape())
a.reshape((1, 6))
print(a)
print(a.shape())

UsageError: Cell magic `%%micropython` not found.
